In [8]:
import sqlite3
import random
import time


class FertilizerControlSystem:
    def __init__(self, db_name="fertilizer_kb.db"):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self._create_tables()
        self._populate_ontology()
        self._populate_rules()

    def _create_tables(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS ontology (
                plant_type TEXT PRIMARY KEY,
                n_sensitivity TEXT,
                description TEXT
            )
        """)
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS rules (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                soil_moisture TEXT,
                plant_type TEXT,
                weather TEXT,
                action TEXT,
                description TEXT
            )
        """)
        self.conn.commit()

    def _populate_ontology(self):
        data = [
            ("Cereal", "High", "Злаковые культуры (пшеница, кукуруза) — высокая потребность в азоте"),
            ("Legume", "Low", "Бобовые (соя, горох) — низкая потребность в азоте, азотфиксирующие растения"),
            ("Vegetable", "Medium", "Овощные культуры — средняя потребность в удобрениях"),
        ]
        self.cursor.executemany("""
            INSERT OR IGNORE INTO ontology(plant_type, n_sensitivity, description)
            VALUES (?, ?, ?)
        """, data)
        self.conn.commit()

    def _populate_rules(self):
        rules = [
            ("Low", "Cereal", "Dry", "HighDose", "Сухо, низкая влажность, злаковые — нужна высокая доза"),
            ("Low", "Vegetable", "Dry", "MediumDose", "Овощные в сухих условиях — средняя доза"),
            ("Low", "Legume", "Dry", "LowDose", "Бобовые в сухих условиях — небольшая доза"),
            ("Optimal", "Cereal", "Normal", "MediumDose", "Злаковые при нормальных условиях — средняя доза"),
            ("Optimal", "Legume", "Normal", "LowDose", "Бобовые при нормальных условиях — малая доза"),
            ("High", "Cereal", "Rainy", "NoFertilizer", "При дожде и высокой влажности — не вносить удобрение"),
            ("High", "Vegetable", "Rainy", "NoFertilizer", "Избыток влаги — не удобрять"),
            ("Low", "Cereal", "Rainy", "LowDose", "Низкая влажность, но дожди — умеренная доза"),
            ("Optimal", "Vegetable", "Normal", "MediumDose", "Оптимальные условия — стандартная доза"),
            ("Optimal", "Legume", "Rainy", "NoFertilizer", "Бобовые и дождь — не удобрять"),
        ]
        self.cursor.executemany("""
            INSERT OR IGNORE INTO rules(soil_moisture, plant_type, weather, action, description)
            VALUES (?, ?, ?, ?, ?)
        """, rules)
        self.conn.commit()

    def fuzzify_moisture(self, value):
        if value < 40:
            return "Low"
        elif 40 <= value <= 70:
            return "Optimal"
        else:
            return "High"

    def fuzzify_weather(self, precipitation):
        if precipitation < 2:
            return "Dry"
        elif 2 <= precipitation <= 8:
            return "Normal"
        else:
            return "Rainy"

    def get_action(self, soil_moisture, plant_type, weather):
        query = """
            SELECT action, description FROM rules
            WHERE soil_moisture=? AND plant_type=? AND weather=?
        """
        self.cursor.execute(query, (soil_moisture, plant_type, weather))
        result = self.cursor.fetchone()
        if result:
            return result[0], result[1]
        else:
            return "NoAction", "Нет подходящего правила — действие не определено."

    def defuzzify_action(self, action):
        mapping = {
            "NoFertilizer": 0,
            "LowDose": 50,
            "MediumDose": 100,
            "HighDose": 150,
            "NoAction": 0
        }
        return mapping.get(action, 0)

    def simulate(self, plant_type="Cereal", steps=10):
        print(f"\n=== Симуляция для типа растения: {plant_type} ===\n")
        soil_moisture_val = random.uniform(30, 70)
        for step in range(1, steps + 1):
            precipitation_val = random.uniform(0, 10)
            moisture = self.fuzzify_moisture(soil_moisture_val)
            weather = self.fuzzify_weather(precipitation_val)
            action, explanation = self.get_action(moisture, plant_type, weather)
            dose = self.defuzzify_action(action)
            print(f"ШАГ {step}")
            print(f"  Влажность почвы: {soil_moisture_val:.1f}% → {moisture}")
            print(f"  Осадки: {precipitation_val:.1f} мм → {weather}")
            print(f"  Действие: {action} (доза: {dose} кг/га)")
            print(f"  Обоснование: {explanation}\n")
            if action == "HighDose":
                soil_moisture_val += random.uniform(2, 5)
            elif action == "MediumDose":
                soil_moisture_val += random.uniform(-2, 2)
            elif action == "LowDose":
                soil_moisture_val -= random.uniform(1, 3)
            else:
                soil_moisture_val += random.uniform(-1, 1)
            soil_moisture_val = max(5, min(95, soil_moisture_val))
            time.sleep(0.3)

    def close(self):
        self.conn.close()


if __name__ == "__main__":
    system = FertilizerControlSystem()
    try:
        system.simulate(plant_type="Cereal", steps=8)
        system.simulate(plant_type="Legume", steps=6)
        system.simulate(plant_type="Vegetable", steps=6)
    finally:
        system.close()



=== Симуляция для типа растения: Cereal ===

ШАГ 1
  Влажность почвы: 46.0% → Optimal
  Осадки: 9.5 мм → Rainy
  Действие: NoAction (доза: 0 кг/га)
  Обоснование: Нет подходящего правила — действие не определено.

ШАГ 2
  Влажность почвы: 45.1% → Optimal
  Осадки: 1.7 мм → Dry
  Действие: NoAction (доза: 0 кг/га)
  Обоснование: Нет подходящего правила — действие не определено.

ШАГ 3
  Влажность почвы: 45.8% → Optimal
  Осадки: 6.2 мм → Normal
  Действие: MediumDose (доза: 100 кг/га)
  Обоснование: Злаковые при нормальных условиях — средняя доза

ШАГ 4
  Влажность почвы: 44.3% → Optimal
  Осадки: 3.9 мм → Normal
  Действие: MediumDose (доза: 100 кг/га)
  Обоснование: Злаковые при нормальных условиях — средняя доза

ШАГ 5
  Влажность почвы: 45.9% → Optimal
  Осадки: 1.1 мм → Dry
  Действие: NoAction (доза: 0 кг/га)
  Обоснование: Нет подходящего правила — действие не определено.

ШАГ 6
  Влажность почвы: 46.9% → Optimal
  Осадки: 5.3 мм → Normal
  Действие: MediumDose (доза: 100 кг/га)